# Joint Image & Text Embeddings

This example shows how SentenceTransformer can be used to map images and texts to the same vector space. 

As model, we use the [OpenAI CLIP Model](https://github.com/openai/CLIP), which was trained on a large set of images and image alt texts.

As a source for fotos, we use the [Unsplash Dataset Lite](https://unsplash.com/data), which contains about 25k images. See the [License](https://unsplash.com/license) about the Unsplash images. 

Note: 25k images is rather small. If you search for really specific terms, the chance are high that no such photo exist in the collection.

## Installation
First we need to install sentence-transformers

In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 5.8 MB 19.5 MB/s 
     |████████████████████████████████| 1.3 MB 70.7 MB/s 
     |████████████████████████████████| 182 kB 92.5 MB/s 
     |████████████████████████████████| 7.6 MB 76.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a08f202ac87c115c88a3b6ca866ea45c0cf1ed6b9e89165cbcb0009ffb48e3b6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


## Load CLIP Model
Next we load the CLIP model using SentenceTransformer. The model is downloaded automatically.

In [2]:
import sentence_transformers
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import glob
import torch
import pickle
import zipfile
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm.autonotebook import tqdm
torch.set_num_threads(4)

#First, we load the respective CLIP model
model = SentenceTransformer('clip-ViT-B-32')



Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/525k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/604 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/961k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

## Google Drive Mount
구글 드라이브에 연동하기


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


### image, text encoding  & cosine similarity using CLIP encoder
image_file_dir : image encoder 에 넣을 파일 디렉토리

text_contents : text encoder 에 넣은 input text

cosine similarity 가 출력될 것임.

In [4]:
'''
image_file_dir = "/content/drive/MyDrive/00000-3951127215-long shot architectural photography of office interior design, a rkrn design chair on the concrete floor, office furniture, larg.png"
text_contents = "I like pizza. What do you think?"

#Encode an image
img_emb = model.encode(Image.open(image_file_dir))

#Encode text descriptions
text_emb = model.encode([text_contents])

#Compute cosine similarities
cos_scores = util.cos_sim(img_emb, text_emb)
print("cosine similarity with the text, \""+text_contents+"\" : ", round(float(cos_scores),4))
'''

'\nimage_file_dir = "/content/drive/MyDrive/00000-3951127215-long shot architectural photography of office interior design, a rkrn design chair on the concrete floor, office furniture, larg.png"\ntext_contents = "I like pizza. What do you think?"\n\n#Encode an image\nimg_emb = model.encode(Image.open(image_file_dir))\n\n#Encode text descriptions\ntext_emb = model.encode([text_contents])\n\n#Compute cosine similarities\ncos_scores = util.cos_sim(img_emb, text_emb)\nprint("cosine similarity with the text, ""+text_contents+"" : ", round(float(cos_scores),4))\n'

In [5]:
'''
image_file_dir = "/content/drive/MyDrive/00000-3951127215-long shot architectural photography of office interior design, a rkrn design chair on the concrete floor, office furniture, larg.png"
text_contents = "A red heart shape chair."

#Encode an image
img_emb = model.encode(Image.open(image_file_dir))

#Encode text descriptions
text_emb = model.encode([text_contents])

#Compute cosine similarities
cos_scores = util.cos_sim(img_emb, text_emb)
print("cosine similarity with the text, \""+text_contents+"\" : ", round(float(cos_scores),4))
'''

'\nimage_file_dir = "/content/drive/MyDrive/00000-3951127215-long shot architectural photography of office interior design, a rkrn design chair on the concrete floor, office furniture, larg.png"\ntext_contents = "A red heart shape chair."\n\n#Encode an image\nimg_emb = model.encode(Image.open(image_file_dir))\n\n#Encode text descriptions\ntext_emb = model.encode([text_contents])\n\n#Compute cosine similarities\ncos_scores = util.cos_sim(img_emb, text_emb)\nprint("cosine similarity with the text, ""+text_contents+"" : ", round(float(cos_scores),4))\n'

### text prompt 리스트와 image file directory 리스트 만들기

In [6]:
root_dir= "/content/drive/MyDrive/text_understanding/image_file/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting", #1 번 text prompt
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting", #2 번 text prompt
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting", #3 번 text prompt
]

image_file_dir_list = [
    [
        root_dir+"text1_1.png", #1-1 image file directory
        root_dir+"text1_2.png" #1-2 image file directory
    ],
    [
        root_dir+"text2_1.png", #2-1 image file directory
        root_dir+"text2_2.png" #2-2 image file directory
    ],
    [
        root_dir+"text3_1.png", #3-1 image file directory
        root_dir+"text3_2.png" #3-2 image file directory
    ],
]



### RP(R-precision) 값 계산하기

In [7]:
score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))

RP of this generator model =  0.8333


### 부록

### model 1-1 RP

In [8]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/1-1/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1-1_ (1).png", #1-1 image file directory
        root_dir+"1-1_ (2).png" #1-2 image file directory
    ],
    [
        root_dir+"1-1_ (3).png", #2-1 image file directory
        root_dir+"1-1_ (4).png" #2-2 image file directory
    ],
    [
        root_dir+"1-1_ (5).png", #3-1 image file directory
        root_dir+"1-1_ (6).png" #3-2 image file directory
    ],
    [
        root_dir+"1-1_ (7).png", #4-1 image file directory
        root_dir+"1-1_ (8).png" #4-2 image file directory
    ],
    [
        root_dir+"1-1_ (9).png", #5-1 image file directory
        root_dir+"1-1_ (10).png" #5-2 image file directory
    ],
    [
        root_dir+"1-1_ (11).png", #6-1 image file directory
        root_dir+"1-1_ (12).png" #6-2 image file directory
    ],
    [
        root_dir+"1-1_ (13).png", #7-1 image file directory
        root_dir+"1-1_ (14).png" #7-2 image file directory
    ],
    [
        root_dir+"1-1_ (15).png", #8-1 image file directory
        root_dir+"1-1_ (16).png" #8-2 image file directory
    ],[
        root_dir+"1-1_ (17).png", #9-1 image file directory
        root_dir+"1-1_ (18).png" #9-2 image file directory
    ],
    [
        root_dir+"1-1_ (19).png", #10-1 image file directory
        root_dir+"1-1_ (20).png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.1
success_count =  2
score_count =  20


### model 1-2 RP

In [9]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/1-2/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1-2_ (1).png", #1-1 image file directory
        root_dir+"1-2_ (2).png" #1-2 image file directory
    ],
    [
        root_dir+"1-2_ (3).png", #2-1 image file directory
        root_dir+"1-2_ (4).png" #2-2 image file directory
    ],
    [
        root_dir+"1-2_ (5).png", #3-1 image file directory
        root_dir+"1-2_ (6).png" #3-2 image file directory
    ],
    [
        root_dir+"1-2_ (7).png", #4-1 image file directory
        root_dir+"1-2_ (8).png" #4-2 image file directory
    ],
    [
        root_dir+"1-2_ (9).png", #5-1 image file directory
        root_dir+"1-2_ (10).png" #5-2 image file directory
    ],
    [
        root_dir+"1-2_ (11).png", #6-1 image file directory
        root_dir+"1-2_ (12).png" #6-2 image file directory
    ],
    [
        root_dir+"1-2_ (13).png", #7-1 image file directory
        root_dir+"1-2_ (14).png" #7-2 image file directory
    ],
    [
        root_dir+"1-2_ (15).png", #8-1 image file directory
        root_dir+"1-2_ (16).png" #8-2 image file directory
    ],[
        root_dir+"1-2_ (17).png", #9-1 image file directory
        root_dir+"1-2_ (18).png" #9-2 image file directory
    ],
    [
        root_dir+"1-2_ (19).png", #10-1 image file directory
        root_dir+"1-2_ (20).png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.6
success_count =  12
score_count =  20


### model 1-3 RP

In [10]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/1-3/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1-3_ (1).png", #1-1 image file directory
        root_dir+"1-3_ (2).png" #1-2 image file directory
    ],
    [
        root_dir+"1-3_ (3).png", #2-1 image file directory
        root_dir+"1-3_ (4).png" #2-2 image file directory
    ],
    [
        root_dir+"1-3_ (5).png", #3-1 image file directory
        root_dir+"1-3_ (6).png" #3-2 image file directory
    ],
    [
        root_dir+"1-3_ (7).png", #4-1 image file directory
        root_dir+"1-3_ (8).png" #4-2 image file directory
    ],
    [
        root_dir+"1-3_ (9).png", #5-1 image file directory
        root_dir+"1-3_ (10).png" #5-2 image file directory
    ],
    [
        root_dir+"1-3_ (11).png", #6-1 image file directory
        root_dir+"1-3_ (12).png" #6-2 image file directory
    ],
    [
        root_dir+"1-3_ (13).png", #7-1 image file directory
        root_dir+"1-3_ (14).png" #7-2 image file directory
    ],
    [
        root_dir+"1-3_ (15).png", #8-1 image file directory
        root_dir+"1-3_ (16).png" #8-2 image file directory
    ],[
        root_dir+"1-3_ (17).png", #9-1 image file directory
        root_dir+"1-3_ (18).png" #9-2 image file directory
    ],
    [
        root_dir+"1-3_ (19).png", #10-1 image file directory
        root_dir+"1-3_ (20).png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.1
success_count =  2
score_count =  20


### model 1-4 RP

In [11]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/1-4/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of office interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, wooden floor, large steel windows viewing a garden, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, office furniture, wooden floor, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, concrete floor, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, concrete floor, studio lighting",
    "close-up architectural photography of coffee shop interior design, voxjs interior design pattern of the ceiling, high ceiling with pendant lighting, wooden floor, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1-4_ (1).png", #1-1 image file directory
        root_dir+"1-4_ (2).png" #1-2 image file directory
    ],
    [
        root_dir+"1-4_ (3).png", #2-1 image file directory
        root_dir+"1-4_ (4).png" #2-2 image file directory
    ],
    [
        root_dir+"1-4_ (5).png", #3-1 image file directory
        root_dir+"1-4_ (6).png" #3-2 image file directory
    ],
    [
        root_dir+"1-4_ (7).png", #4-1 image file directory
        root_dir+"1-4_ (8).png" #4-2 image file directory
    ],
    [
        root_dir+"1-4_ (9).png", #5-1 image file directory
        root_dir+"1-4_ (10).png" #5-2 image file directory
    ],
    [
        root_dir+"1-4_ (11).png", #6-1 image file directory
        root_dir+"1-4_ (12).png" #6-2 image file directory
    ],
    [
        root_dir+"1-4_ (13).png", #7-1 image file directory
        root_dir+"1-4_ (14).png" #7-2 image file directory
    ],
    [
        root_dir+"1-4_ (15).png", #8-1 image file directory
        root_dir+"1-4_ (16).png" #8-2 image file directory
    ],[
        root_dir+"1-4_ (17).png", #9-1 image file directory
        root_dir+"1-4_ (18).png" #9-2 image file directory
    ],
    [
        root_dir+"1-4_ (19).png", #10-1 image file directory
        root_dir+"1-4_ (20).png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.45
success_count =  9
score_count =  20


### model 2-1 RP

In [12]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/2-1/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, rkrn design chair on the concrete floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, blue rkrn design chair on the concrete floor, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, blue rkrn design chair on the wooden floor, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the concrete floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the concrete floor, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the wooden floor, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"2-1-1_1.png", #1-1 image file directory
        root_dir+"2-1-1_3.png" #1-2 image file directory
    ],
    [
        root_dir+"2-1-2_1.png", #2-1 image file directory
        root_dir+"2-1-2_3.png" #2-2 image file directory
    ],
    [
        root_dir+"2-1-3_1.png", #3-1 image file directory
        root_dir+"2-1-3_2.png" #3-2 image file directory
    ],
    [
        root_dir+"2-1-4_1.png", #4-1 image file directory
        root_dir+"2-1-4_3.png" #4-2 image file directory
    ],
    [
        root_dir+"2-1-5_1.png", #5-1 image file directory
        root_dir+"2-1-5_2.png" #5-2 image file directory
    ],
    [
        root_dir+"2-1-6_1.png", #6-1 image file directory
        root_dir+"2-1-6_2.png" #6-2 image file directory
    ],
    [
        root_dir+"2-1-7_1.png", #7-1 image file directory
        root_dir+"2-1-7_2.png" #7-2 image file directory
    ],
    [
        root_dir+"2-1-8_1.png", #8-1 image file directory
        root_dir+"2-1-8_2.png" #8-2 image file directory
    ],[
        root_dir+"2-1-9_1.png", #9-1 image file directory
        root_dir+"2-1-9_3.png" #9-2 image file directory
    ],
    [
        root_dir+"2-1-10_1.png", #10-1 image file directory
        root_dir+"2-1-10_2.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.25
success_count =  5
score_count =  20


### model 2-2 RP

In [13]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/2-2/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, rkrn design chair on the concrete floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, blue rkrn design chair on the concrete floor, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, blue rkrn design chair on the wooden floor, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the concrete floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the concrete floor, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the wooden floor, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"2-2-1_1.png", #1-1 image file directory
        root_dir+"2-2-1_3.png" #1-2 image file directory
    ],
    [
        root_dir+"2-2-2_1.png", #2-1 image file directory
        root_dir+"2-2-2_2.png" #2-2 image file directory
    ],
    [
        root_dir+"2-2-3_1.png", #3-1 image file directory
        root_dir+"2-2-3_2.png" #3-2 image file directory
    ],
    [
        root_dir+"2-2-4_1.png", #4-1 image file directory
        root_dir+"2-2-4_2.png" #4-2 image file directory
    ],
    [
        root_dir+"2-2-5_1.png", #5-1 image file directory
        root_dir+"2-2-5_2.png" #5-2 image file directory
    ],
    [
        root_dir+"2-2-6_1.png", #6-1 image file directory
        root_dir+"2-2-6_2.png" #6-2 image file directory
    ],
    [
        root_dir+"2-2-7_2.png", #7-1 image file directory
        root_dir+"2-2-7_3.png" #7-2 image file directory
    ],
    [
        root_dir+"2-2-8_2.png", #8-1 image file directory
        root_dir+"2-2-8_3.png" #8-2 image file directory
    ],[
        root_dir+"2-2-9_1.png", #9-1 image file directory
        root_dir+"2-2-9_3.png" #9-2 image file directory
    ],
    [
        root_dir+"2-2-10_1.png", #10-1 image file directory
        root_dir+"2-2-10_2.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.75
success_count =  15
score_count =  20


### model 2-3 RP

In [17]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/2-3/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, rkrn design chair on the concrete floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, blue rkrn design chair on the concrete floor, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, blue rkrn design chair on the wooden floor, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the concrete floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the concrete floor, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the wooden floor, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"2-3-1/2-3-1_Euler_a.png", #1-1 image file directory
        root_dir+"2-3-1/2-3-1_PLMS.png" #1-2 image file directory
    ],
    [
        root_dir+"2-3-2/2-3-2_Euler_a.png", #2-1 image file directory
        root_dir+"2-3-2/2-3-2_PLMS.png" #2-2 image file directory
    ],
    [
        root_dir+"2-3-3/2-3-3_Euler_a.png", #3-1 image file directory
        root_dir+"2-3-3/2-3-3_PLMS.png" #3-2 image file directory
    ],
    [
        root_dir+"2-3-4/2-3-4_Euler_a.png", #4-1 image file directory
        root_dir+"2-3-4/2-3-4_PLMS.png" #4-2 image file directory
    ],
    [
        root_dir+"2-3-5/2-3-5_DDIM.png", #5-1 image file directory
        root_dir+"2-3-5/2-3-5_PLMS.png" #5-2 image file directory
    ],
    [
        root_dir+"2-3-6/2-3-6_DDIM.png", #6-1 image file directory
        root_dir+"2-3-6/2-3-6_Euler_a.png" #6-2 image file directory
    ],
    [
        root_dir+"2-3-7/2-3-7_Euler_a.png", #7-1 image file directory
        root_dir+"2-3-7/2-3-7_DDIM.png" #7-2 image file directory
    ],
    [
        root_dir+"2-3-8/2-3-8_DDIM.png", #8-1 image file directory
        root_dir+"2-3-8/2-3-8_Euler_a.png" #8-2 image file directory
    ],[
        root_dir+"2-3-9/2-3-9_DDIM.png", #9-1 image file directory
        root_dir+"2-3-9/2-3-9_Euler_a.png" #9-2 image file directory
    ],
    [
        root_dir+"2-3-10/2-3-10_DDIM.png", #10-1 image file directory
        root_dir+"2-3-10/2-3-10_Euler_a.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.75
success_count =  15
score_count =  20


### model 2-4 RP

In [18]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/2-4/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, rkrn design chair on the concrete floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, rkrn design chair on the wooden floor, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, blue rkrn design chair on the concrete floor, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, blue rkrn design chair on the wooden floor, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the concrete floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, rkrn design chair on the wooden floor, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the concrete floor, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, blue rkrn design chair on the wooden floor, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1/00135-362999142-long shot arch___.png", #1-1 image file directory
        root_dir+"1/00136-1522152588-long shot arc___.png" #1-2 image file directory
    ],
    [
        root_dir+"2/00137-756588419-long shot arch___.png", #2-1 image file directory
        root_dir+"2/00139-3410877700-long shot arc___.png" #2-2 image file directory
    ],
    [
        root_dir+"3/00140-1010494674-long shot arc___.png", #3-1 image file directory
        root_dir+"3/00141-1351072140-long shot arc___.png" #3-2 image file directory
    ],
    [
        root_dir+"4/00143-2923625423-close-up arch___.png", #4-1 image file directory
        root_dir+"4/00144-1994153045-close-up arch___.png" #4-2 image file directory
    ],
    [
        root_dir+"5/00146-3453585270-close-up arch___.png", #5-1 image file directory
        root_dir+"5/00147-2330886735-close-up arch___.png" #5-2 image file directory
    ],
    [
        root_dir+"6/00149-1672985191-long shot arc___.png", #6-1 image file directory
        root_dir+"6/00151-22795-long shot architec___.png" #6-2 image file directory
    ],
    [
        root_dir+"7/00152-1702888460-long shot arc___.png", #7-1 image file directory
        root_dir+"7/00153-927769743-long shot arch___.png" #7-2 image file directory
    ],
    [
        root_dir+"8/00156-3938548952-long shot arc___.png", #8-1 image file directory
        root_dir+"8/00157-654823179-long shot arch___.png" #8-2 image file directory
    ],[
        root_dir+"9/00158-254637671-close-up archi___.png", #9-1 image file directory
        root_dir+"9/00159-886109407-close-up archi___.png" #9-2 image file directory
    ],
    [
        root_dir+"10/00161-3722086334-close-up arch___.png", #10-1 image file directory
        root_dir+"10/00162-3858554204-close-up arch___.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.3
success_count =  6
score_count =  20


### model 3-1 RP

In [19]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/3-1/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, green fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, green fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1/00005-3683598537-long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, large steel windo.png", #1-1 image file directory
        root_dir+"1/00006-3719661797-long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, large steel windo.png" #1-2 image file directory
    ],
    [
        root_dir+"2/00032-789020764-long shot arch___.png", #2-1 image file directory
        root_dir+"2/00034-4024130646-long shot arc___.png" #2-2 image file directory
    ],
    [
        root_dir+"3/00007-4077991793-long shot arc___.png", #3-1 image file directory
        root_dir+"3/00009-3920652563-long shot arc___.png" #3-2 image file directory
    ],
    [
        root_dir+"4/00011-3807375964-close-up arch___.png", #4-1 image file directory
        root_dir+"4/00012-2724216381-close-up arch___.png" #4-2 image file directory
    ],
    [
        root_dir+"5/00013-3899012468-close-up arch___.png", #5-1 image file directory
        root_dir+"5/00014-1923538981-close-up arch___.png" #5-2 image file directory
    ],
    [
        root_dir+"6/00017-2001772613-long shot arc___.png", #6-1 image file directory
        root_dir+"6/00018-3009878168-long shot arc___.png" #6-2 image file directory
    ],
    [
        root_dir+"7/00035-154256006-long shot arch___.png", #7-1 image file directory
        root_dir+"7/00037-1418924398-long shot arc___.png" #7-2 image file directory
    ],
    [
        root_dir+"8/00020-3756887130-long shot arc___.png", #8-1 image file directory
        root_dir+"8/00022-1541201604-long shot arc___.png" #8-2 image file directory
    ],[
        root_dir+"9/00025-2125889587-close-up arch___.png", #9-1 image file directory
        root_dir+"9/00026-3978111958-close-up arch___.png" #9-2 image file directory
    ],
    [
        root_dir+"10/00028-2930159058-close-up arch___.png", #10-1 image file directory
        root_dir+"10/00030-2986554681-close-up arch___.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.55
success_count =  11
score_count =  20


### model 3-2 RP

In [21]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/3-2/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, green fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, green fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1/00040-2590794145-long shot arc___.png", #1-1 image file directory
        root_dir+"1/00041-172881479-long shot arch___.png" #1-2 image file directory
    ],
    [
        root_dir+"2/00042-211895531-long shot arch___.png", #2-1 image file directory
        root_dir+"2/00043-2480587696-long shot arc___.png" #2-2 image file directory
    ],
    [
        root_dir+"3/00046-2586204894-long shot arc___.png", #3-1 image file directory
        root_dir+"3/00047-1242978361-long shot arc___.png" #3-2 image file directory
    ],
    [
        root_dir+"4/00048-1194207558-close-up arch___.png", #4-1 image file directory
        root_dir+"4/00049-3452264368-close-up arch___.png" #4-2 image file directory
    ],
    [
        root_dir+"5/00052-4137494400-close-up arch___.png", #5-1 image file directory
        root_dir+"5/00053-3875590422-close-up arch___.png" #5-2 image file directory
    ],
    [
        root_dir+"6/00055-1077490265-long shot arc___.png", #6-1 image file directory
        root_dir+"6/00056-2868882908-long shot arc___.png" #6-2 image file directory
    ],
    [
        root_dir+"7/00058-332779521-long shot arch___.png", #7-1 image file directory
        root_dir+"7/00060-4155779891-long shot arc___.png" #7-2 image file directory
    ],
    [
        root_dir+"8/00061-3949982503-long shot arc___.png", #8-1 image file directory
        root_dir+"8/00063-1119318861-long shot arc___.png" #8-2 image file directory
    ],[
        root_dir+"9/00064-1494929992-close-up arch___.png", #9-1 image file directory
        root_dir+"9/00066-1947210652-close-up arch___.png" #9-2 image file directory
    ],
    [
        root_dir+"10/00068-3437812894-close-up arch___.png", #10-1 image file directory
        root_dir+"10/00069-570465955-close-up archi___.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.4
success_count =  8
score_count =  20


### model 3-3 RP

In [22]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/3-3/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, green fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, green fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1/00070-589568448-long shot arch___.png", #1-1 image file directory
        root_dir+"1/00071-1415617394-long shot arc___.png" #1-2 image file directory
    ],
    [
        root_dir+"2/00073-4131586743-long shot arc___.png", #2-1 image file directory
        root_dir+"2/00074-4003543489-long shot arc___.png" #2-2 image file directory
    ],
    [
        root_dir+"3/00077-2491135164-long shot arc___.png", #3-1 image file directory
        root_dir+"3/00079-780037335-long shot arch___.png" #3-2 image file directory
    ],
    [
        root_dir+"4/00080-2752739066-close-up arch___.png", #4-1 image file directory
        root_dir+"4/00081-1201793668-close-up arch___.png" #4-2 image file directory
    ],
    [
        root_dir+"5/00085-3823281866-close-up arch___.png", #5-1 image file directory
        root_dir+"5/00086-1620156666-close-up arch___.png" #5-2 image file directory
    ],
    [
        root_dir+"6/00087-370022275-long shot arch___.png", #6-1 image file directory
        root_dir+"6/00088-2378535434-long shot arc___.png" #6-2 image file directory
    ],
    [
        root_dir+"7/00091-2514705090-long shot arc___.png", #7-1 image file directory
        root_dir+"7/00092-2202507624-long shot arc___.png" #7-2 image file directory
    ],
    [
        root_dir+"8/00094-889524411-long shot arch___.png", #8-1 image file directory
        root_dir+"8/00096-2058034921-long shot arc___.png" #8-2 image file directory
    ],[
        root_dir+"9/00097-3578850576-close-up arch___.png", #9-1 image file directory
        root_dir+"9/00099-3445590913-close-up arch___.png" #9-2 image file directory
    ],
    [
        root_dir+"10/00101-3069017121-close-up arch___.png", #10-1 image file directory
        root_dir+"10/00102-321338204-close-up archi___.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.2
success_count =  4
score_count =  20


### model 3-4 RP

In [23]:


root_dir= "/content/drive/MyDrive/text_understanding/image_file/3-4/" # <- 여기 디렉토리에 생성된 이미지 파일을 저장해야 함.


text_prompt_list = [
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #1 번 text prompt
    "long shot architectural photography of office interior design, green fhrh logo attached to a wall, office furniture, large steel windows viewing a garden, modern design, studio lighting", #2 번 text prompt
    "long shot architectural photography of office interior design, fhrh logo attached to a wall, office furniture, high ceiling with pendant lighting, modern design, studio lighting", #3 번 text prompt
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, large steel windows viewing a garden, modern design, studio lighting",
    "close-up architectural photography of office interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, modern design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, green fhrh logo attached to a wall, small tea tables and chairs, high ceiling with pendant lighting, natural design, studio lighting",
    "long shot architectural photography of coffee shop interior design, fhrh logo attached to a wall, small tea tables and chairs, large steel windows viewing a garden, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, high ceiling with pendant lighting, natural design, studio lighting",
    "close-up architectural photography of coffee shop interior design, fhrh logo attached to a wall, large steel windows viewing a garden, natural design, studio lighting",
]

image_file_dir_list = [
    [
        root_dir+"1/00104-2439012357-long shot arc___.png", #1-1 image file directory
        root_dir+"1/00105-3089134991-long shot arc___.png" #1-2 image file directory
    ],
    [
        root_dir+"2/00106-3580589398-long shot arc___.png", #2-1 image file directory
        root_dir+"2/00108-2456572860-long shot arc___.png" #2-2 image file directory
    ],
    [
        root_dir+"3/00109-3558340256-long shot arc___.png", #3-1 image file directory
        root_dir+"3/00110-2881700854-long shot arc___.png" #3-2 image file directory
    ],
    [
        root_dir+"4/00112-2108964804-close-up arch___.png", #4-1 image file directory
        root_dir+"4/00113-527423438-close-up archi___.png" #4-2 image file directory
    ],
    [
        root_dir+"5/00115-3946468153-close-up arch___.png", #5-1 image file directory
        root_dir+"5/00118-2070146341-close-up arch___.png" #5-2 image file directory
    ],
    [
        root_dir+"6/00119-3879454928-long shot arc___.png", #6-1 image file directory
        root_dir+"6/00120-568850953-long shot arch___.png" #6-2 image file directory
    ],
    [
        root_dir+"7/00123-1677067743-long shot arc___.png", #7-1 image file directory
        root_dir+"7/00124-2991318066-long shot arc___.png" #7-2 image file directory
    ],
    [
        root_dir+"8/00125-3365755039-long shot arc___.png", #8-1 image file directory
        root_dir+"8/00127-1459543513-long shot arc___.png" #8-2 image file directory
    ],[
        root_dir+"9/00129-3659525319-close-up arch___.png", #9-1 image file directory
        root_dir+"9/00130-3711359895-close-up arch___.png" #9-2 image file directory
    ],
    [
        root_dir+"10/00131-2771332167-close-up arch___.png", #10-1 image file directory
        root_dir+"10/00132-3980516991-close-up arch___.png" #10-2 image file directory
    ],
]

score_count = 0
success_count = 0
for image_index, x in enumerate(image_file_dir_list):
  for image_file_dir in x:
    score_count = score_count + 1

    score_vec = [0.0] * len(text_prompt_list)
    for text_index, text_prompt in enumerate(text_prompt_list):
      
      img_emb = model.encode(Image.open(image_file_dir))
      text_emb = model.encode([text_prompt])

      cos_scores = util.cos_sim(img_emb, text_emb)

      score_vec[text_index] = cos_scores
    max_index = score_vec.index(max(score_vec))
    if max_index == image_index:
      success_count  = success_count + 1

RP = float(success_count) / float(score_count)

print("RP of this generator model = ",round(RP,4))
print("success_count = ", success_count)
print("score_count = ", score_count)

RP of this generator model =  0.2
success_count =  4
score_count =  20
